# Assignment 3

In [1]:
import pandas as pd
import numpy as np

# Load MovieLens 100K dataset into a dataframe of pandas
names = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('ml-100k/u.data', sep='\t', names=names)
df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [2]:
# Select 500 most active users and 500 most active items from the dataset
n_most_active_users = 500
n_most_active_items = 500

user_ids = df.groupby('user_id').count().sort_values(by='rating', ascending=False).head(n_most_active_users).index
item_ids = df.groupby('item_id').count().sort_values(by='rating', ascending=False).head(n_most_active_items).index
df = df[(df['user_id'].isin(user_ids)) & (df['item_id'].isin(item_ids))]

In [3]:
# Map new internal ID for items
i_ids = df['item_id'].unique().tolist()
item_dict = dict(zip(i_ids, [i for i in range(len(i_ids))]))
df['item_id'] = df['item_id'].map(item_dict)

# Split Dataset

In [4]:
# The number of training users and active users
n_training_users = 300
n_active_users = n_most_active_users - n_training_users

# The number of GIVEN ratings for active users
GIVEN = 20

# Randomly select users from the most active users as training set
random_uids = np.random.choice(df.user_id.unique(), n_training_users, replace=False)
train_df = df[df['user_id'].isin(random_uids)]
# Map new internal ID for all users in the training set
u_ids = train_df['user_id'].unique().tolist()
user_dict = dict(zip(u_ids, [i for i in range(len(u_ids))]))
train_df['user_id'] = train_df['user_id'].map(user_dict)

# The rest of users are active users for testing
remain_df = df[~df['user_id'].isin(random_uids)]
# Map new internal ID for all active users
u_ids = remain_df['user_id'].unique().tolist()
user_dict = dict(zip(u_ids, [i for i in range(len(u_ids))]))
remain_df['user_id'] = remain_df['user_id'].map(user_dict)

# Randomly select GIVEN ratings for active users
active_df = remain_df.groupby('user_id').sample(n=GIVEN, random_state=1024)

test_df = remain_df[~remain_df.index.isin(active_df.index)]

<ipython-input-4-fb140fbf3b12>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['user_id'] = train_df['user_id'].map(user_dict)
<ipython-input-4-fb140fbf3b12>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  remain_df['user_id'] = remain_df['user_id'].map(user_dict)


In [5]:
# Convert the format of datasets to matrices
df_zeros = pd.DataFrame({'user_id': np.tile(np.arange(0, n_training_users), n_most_active_items), 'item_id': np.repeat(np.arange(0, n_most_active_items), n_training_users), 'rating': 0})
train_ds = df_zeros.merge(train_df, how='left', on=['user_id', 'item_id']).fillna(0.).pivot_table(values='rating_y', index='user_id', columns='item_id')

df_zeros = pd.DataFrame({'user_id': np.tile(np.arange(0, n_active_users), n_most_active_items), 'item_id': np.repeat(np.arange(0, n_most_active_items), n_active_users), 'rating': 0})
active_ds = df_zeros.merge(active_df, how='left', on=['user_id', 'item_id']).fillna(0.).pivot_table(values='rating_y', index='user_id', columns='item_id')
test_ds = df_zeros.merge(test_df, how='left', on=['user_id', 'item_id']).fillna(0.).pivot_table(values='rating_y', index='user_id', columns='item_id')

train_ds, active_ds, test_ds

(item_id  0    1    2    3    4    5    6    7    8    9    ...  490  491  492  \
 user_id                                                    ...                  
 0        3.0  0.0  0.0  0.0  0.0  0.0  0.0  4.0  0.0  0.0  ...  0.0  0.0  0.0   
 1        0.0  2.0  0.0  4.0  0.0  4.0  4.0  0.0  0.0  2.0  ...  0.0  4.0  4.0   
 2        4.0  0.0  0.0  2.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
 3        4.0  0.0  5.0  0.0  0.0  3.0  4.0  2.0  0.0  2.0  ...  0.0  2.0  0.0   
 4        4.0  0.0  5.0  0.0  1.0  0.0  3.0  2.0  0.0  0.0  ...  0.0  0.0  0.0   
 ...      ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
 295      4.0  0.0  4.0  0.0  0.0  4.0  3.0  4.0  0.0  0.0  ...  0.0  0.0  0.0   
 296      0.0  0.0  0.0  0.0  0.0  0.0  0.0  5.0  4.0  5.0  ...  0.0  0.0  4.0   
 297      0.0  0.0  5.0  4.0  0.0  1.0  0.0  0.0  0.0  1.0  ...  0.0  0.0  0.0   
 298      0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  3.0  4.0  0.0   
 299      0.0  0

In [6]:
# Predicting All Missing Data in training set
imputed_train_ds = train_ds.values.copy()

# Your implementation to predict the missing values
(Put all your implementation for your algorithm in the following cell only to handle the missing values; )

In [7]:
## Put all your implementation for your solutioin in this cell only to predict the missing values; 
## NOTE 1: DO NOT change anything in the rest of the cells in this framework, 
## otherwise the changes might cause errors and make your implementation invalid.

## Note 2: 
## The user-item rating matrix is imputed_train_ds, 
## and the missing values are those 0s in imputed_train_ds. 
## You are required to predict them by using the solution in the given report. 

## The following parameters are required in the given report, 
## which is named "Effective Missing Data Prediction for Collaborative Filtering", 
## and you will need to use them. But, please do not change their values. 
import math
LAMBDA = 0.7    # λ
GAMMA = 10      # γ
DELTA = 10      # δ
ITA = 0.7       # η
THETA = 0.7     # θ
EPSILON = 1e-9
# Predicting All Missing Data in training set
imputed_train_ds = train_ds.values.copy()
# For users
def simUser(data,para,threshold):
    load_ar = np.array(data)
    lie,hang=load_ar.shape
    # Save user's mean values
    user_mean=[0 for k in range(lie)]
    index=[k for k in range(hang)]
    load_pc=np.zeros((lie,lie))
    # Calculate the weighting matrix
    weight=np.zeros((lie,lie))
    for i in range(lie):
        for j in range(i,lie):
            if i==j:
                load_pc[i][j]=1.0
                weight[i][j]=min((load_ar[i,:]!=0).sum(),para)*1.0/para
            else:
                # User common evaluation's index
                i_index=np.array(index)[(load_ar[i]!=0)]
                j_index=np.array(index)[(load_ar[j]!=0)]
                commom=list(set(i_index)&set(j_index))
                mean_i=sum(load_ar[i])*1.0/len(i_index)
                user_mean[i]=mean_i
                mean_j=sum(load_ar[j])*1.0/len(j_index)
                A=0
                B=0
                C=0
                for each in commom:
                    A+=(load_ar[i][each]-mean_i)*(load_ar[j][each]-mean_j)
                    B+=(load_ar[i][each]-mean_i)**2
                    C+=(load_ar[j][each]-mean_j)**2
                if len(commom)!=0:
                    load_pc[i][j]=A/(math.sqrt(B)*math.sqrt(C))
                    load_pc[j][i]=load_pc[i][j]
                n=len(commom)
                weight[i][j]=min(n,para)*1.0/para
                weight[j][i]=weight[i][j]
    User_User=weight*load_pc
    return User_User,user_mean
# For item
def simItem(data,para,threshold):
    load_ar = np.array(data)
    load_ar=load_ar.T
    lie, hang = load_ar.shape
    # Save item's mean value
    item_mean = [0 for k in range(lie)]
    index = [k for k in range(hang)]
    load_pc = np.zeros((lie, lie))
    # Calculate the weight matrix
    weight = np.zeros((lie, lie))
    for i in range(lie):
        for j in range(i, lie):
            if i == j:
                load_pc[i][j] = 1.0
                weight[i][j] = min((load_ar[i, :] != 0).sum(), para) * 1.0 / para
            else:
                # Item common evaluation 's index
                i_index = np.array(index)[(load_ar[i] != 0)]
                j_index = np.array(index)[(load_ar[j] != 0)]
                commom = list(set(i_index) & set(j_index))
                mean_i = sum(load_ar[i]) * 1.0 / len(i_index)
                item_mean[i]=mean_i
                mean_j = sum(load_ar[j]) * 1.0 / len(j_index)
                A = 0
                B = 0
                C = 0
                for each in commom:
                    A += (load_ar[i][each] - mean_i) * (load_ar[j][each] - mean_j)
                    B += (load_ar[i][each] - mean_i) ** 2
                    C += (load_ar[j][each] - mean_j) ** 2
                if len(commom) != 0:
                    load_pc[i][j] = A / (math.sqrt(B) * math.sqrt(C))
                    load_pc[j][i] = load_pc[i][j]
                n = len(commom)
                weight[i][j] = min(n, para) * 1.0 / para
                weight[j][i] = weight[i][j]
    item_item = weight * load_pc
    # print(item_item)
    return item_item,item_mean

LAMBDA = 0.7    # λ
GAMMA = 10      # γ
DELTA = 10      # δ
ITA = 0.7       # η
THETA = 0.7     # θ
EPSILON = 1e-9
# For the missing values:
# Firstly, will generate similar users and similar users will be calculated by the PCC for users. 
# And will generate similar items and similar items will be calculated by the PCC for items. 
# Only if there are neither similar users nor similar items,the missing values will be still 'missing'
user2user,user_mean=simUser(train_ds,GAMMA,ITA)
u_index=[k for k in range(len(user_mean))]
similaryUser=user2user>ITA
item2item,item_mean=simItem(train_ds,DELTA,THETA)
i_index=[k for k in range(len(item_mean))]
similaryItem=item2item>THETA
# Fill in missing values
# Original evaluation matrix is train_ds
def predict_values(data,type):
    rating_matrix=np.array(data)
    for u in range(len(rating_matrix)):
        for i in range(len(rating_matrix[0])):
            # Using ==0 when evaluation part is missing,it will need to fill in
            if rating_matrix[u][i]==0:
                # Get the similar users and items
                sim_user_u=np.array(u_index)[similaryUser[u]]
                sim_item_i=np.array(i_index)[similaryItem[i]]
                u_score = 0
                i_score=0
                # Based on the users' score
                if len(sim_user_u)>1:
                    A=0
                    B=0
                    for each in sim_user_u:
                        if each!=u:
                            A+=user2user[u][each]*(rating_matrix[each][i]-user_mean[each])
                            B+=user2user[u][each]
                    u_score+=user_mean[u]+A*1.0/B
                # Based on the items' score
                if len(sim_item_i)>1:
                    A=0
                    B=0
                    for each in sim_item_i:
                        if each!=i:
                            A+=item2item[i][each]*(rating_matrix[u][each]-item_mean[each])
                            B+=item2item[i][each]
                # If there are neither similar users nor similar items
                if u_score*i_score==0:
                    if (u_score+i_score)!=0:
                        rating_matrix[u][i]=u_score+i_score
                    else:
                        if type=='active':
                            rating_matrix[u][i]=LAMBDA*user_mean[u]+(1-LAMBDA)*item_mean[i]
                else:
                    rating_matrix[u][i]=LAMBDA*u_score+(1-LAMBDA)*i_score
    return rating_matrix
imputed_train_ds=predict_values(imputed_train_ds,'active')
imputed_train_ds=pd.DataFrame(imputed_train_ds)


<ipython-input-7-5325d61d222a>:92: RuntimeWarning: invalid value encountered in double_scalars
  load_pc[i][j] = A / (math.sqrt(B) * math.sqrt(C))


# Evaluation

### Compute Pearson Correlation Coefficient of All Pairs of Items between active set and imputed training set

In [8]:
imputed_train_ds = pd.DataFrame(imputed_train_ds)
imputed_train_ds

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,3.000000,3.472449,3.723404,3.592857,3.506250,3.440000,3.665789,4.000000,3.381250,3.529641,...,3.312500,3.342683,3.578261,3.661111,3.395652,3.080769,4.000000,3.478571,3.000000,2.450000
1,3.813407,2.000000,3.845227,4.000000,3.628073,4.000000,4.000000,3.655694,3.503073,2.000000,...,3.434323,4.000000,4.000000,3.782934,3.000000,3.000000,3.237448,3.600395,3.415573,2.571823
2,4.000000,3.796199,4.047154,2.000000,3.830000,3.763750,3.989539,3.857621,3.705000,3.853391,...,3.636250,3.666433,3.902011,3.984861,3.719402,3.404519,3.439375,3.802321,3.617500,2.773750
3,4.000000,3.397730,5.000000,3.518138,3.431531,3.000000,4.000000,2.000000,3.306531,2.000000,...,3.237781,2.000000,3.503542,3.586392,3.320933,3.006050,3.040906,4.000000,3.219031,2.375281
4,4.000000,3.563926,5.000000,3.684334,1.000000,3.531477,3.000000,2.000000,3.472727,3.621118,...,3.403977,3.434160,3.669738,4.000000,1.000000,3.172247,3.207102,3.570049,3.385227,2.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,4.000000,3.685730,4.000000,3.806138,3.719531,4.000000,3.000000,4.000000,3.594531,3.742922,...,3.525781,3.555964,3.791542,3.874392,3.608933,3.294050,3.328906,3.691853,3.507031,2.663281
296,2.126935,0.120404,0.120404,0.120404,0.120404,0.120404,0.120404,5.000000,4.000000,5.000000,...,0.720092,0.120404,4.000000,0.120404,4.000000,2.000000,3.000000,0.706477,0.120404,0.120404
297,3.202482,2.983346,5.000000,4.000000,3.017147,1.000000,3.176687,3.044768,2.892147,1.000000,...,2.823397,2.853580,3.089158,3.172009,2.906550,2.591667,2.626522,2.989469,1.000000,1.960897
298,3.707493,1.000000,3.739313,3.608766,3.522159,3.455909,3.681699,3.549780,3.397159,3.545550,...,3.000000,4.000000,3.594170,3.677020,3.411561,3.096678,3.131534,3.494481,3.309659,2.465909


In [9]:
active_user_pearson_corr = np.zeros((active_ds.shape[0], train_ds.shape[0]))

# Compute Pearson Correlation Coefficient of All Pairs of Users between active set and imputed training set
for i, user_i_vec in enumerate(active_ds.values):
    for j, user_j_vec in enumerate(imputed_train_ds.values):
        
        # ratings corated by the current pair od users
        mask_i = user_i_vec > 0
        mask_j = user_j_vec > 0

        # corrated item index, skip if there are no corrated ratings
        corrated_index = np.intersect1d(np.where(mask_i), np.where(mask_j))
        if len(corrated_index) == 0:
            continue

        # average value of user_i_vec and user_j_vec
        mean_user_i = np.sum(user_i_vec) / (np.sum(np.clip(user_i_vec, 0, 1)) + EPSILON)
        mean_user_j = np.sum(user_j_vec) / (np.sum(np.clip(user_j_vec, 0, 1)) + EPSILON)

        # compute pearson corr
        user_i_sub_mean = user_i_vec[corrated_index] - mean_user_i
        user_j_sub_mean = user_j_vec[corrated_index] - mean_user_j

        r_ui_sub_r_i_sq = np.square(user_i_sub_mean)
        r_uj_sub_r_j_sq = np.square(user_j_sub_mean)

        r_ui_sum_sqrt = np.sqrt(np.sum(r_ui_sub_r_i_sq))
        r_uj_sum_sqrt = np.sqrt(np.sum(r_uj_sub_r_j_sq))

        sim = np.sum(user_i_sub_mean * user_j_sub_mean) / (r_ui_sum_sqrt * r_uj_sum_sqrt + EPSILON)

        # significance weighting
        weighted_sim = (min(len(corrated_index), GAMMA) / GAMMA) * sim

        active_user_pearson_corr[i][j] = weighted_sim

active_user_pearson_corr

array([[ 0.15297772,  0.06581631,  0.15688735, ...,  0.1923687 ,
        -0.02310991,  0.17750324],
       [ 0.31341232,  0.09469369, -0.28086223, ...,  0.20444811,
        -0.13359192, -0.0419301 ],
       [-0.04299883,  0.45597917,  0.13703922, ...,  0.3567764 ,
         0.54339542,  0.24159328],
       ...,
       [-0.22425133,  0.27094149,  0.22220861, ...,  0.08732157,
         0.26141798,  0.07121421],
       [ 0.07154787, -0.07056512, -0.05950958, ...,  0.1512259 ,
         0.34851855,  0.28920255],
       [ 0.28989225,  0.3066502 ,  0.25209015, ..., -0.14435992,
         0.35720447, -0.29808048]])

## Predict Ratings of Testing Set

In [10]:
K = 10

test_ds_pred = np.zeros_like(test_ds.values)

for (i, j), rating in np.ndenumerate(test_ds.values):

    if rating > 0:

        sim_user_ids = np.argsort(active_user_pearson_corr[i])[-1:-(K + 1):-1]

        #==================user-based==================#
        # the coefficient values of similar users
        sim_val = active_user_pearson_corr[i][sim_user_ids]

        # the average value of the current user's ratings
        sim_users = imputed_train_ds.values[sim_user_ids]
        user_mean = np.sum(active_ds.values[i]) / (np.sum(np.clip(active_ds.values[i], 0, 1)) + EPSILON)
        sim_user_mean = np.sum(sim_users, axis=1) / (np.sum(np.clip(sim_users, 0, 1), axis=1) + EPSILON)

        # select the users who rated item j
        mask_rated_j = sim_users[:, j] > 0
        
        # sim(u, v) * (r_vj - mean_v)
        sim_r_sum_mean = sim_val[mask_rated_j] * (sim_users[mask_rated_j, j] - sim_user_mean[mask_rated_j])
        
        user_based_pred = user_mean + np.sum(sim_r_sum_mean) / (np.sum(sim_val[mask_rated_j]) + EPSILON)
        user_based_pred = np.clip(user_based_pred, 0, 5)

        test_ds_pred[i][j] = user_based_pred
        
test_ds_pred


array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [3.62769745, 0.        , 4.06601433, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [4.30579592, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

## Compute MAE and RMSE

In [11]:
# MAE
MAE = np.sum(np.abs(test_ds_pred - test_ds.values)) / np.sum(np.clip(test_ds.values, 0, 1))

# RMSE
RMSE = np.sqrt(np.sum(np.square(test_ds_pred - test_ds.values)) / np.sum(np.clip(test_ds.values, 0, 1)))

print("MAE: {}, RMSE: {}" .format(MAE, RMSE))

MAE: 0.7663373845314435, RMSE: 0.9805146138857551
